# Carga y Análisis: Análisis Descriptivo
    
**Objetivo:** Comprender la distribución y el comportamiento de los clientes mediante métricas estadísticas (media, mediana, desviación estándar).


In [1]:
#Importación de librerías
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

In [2]:
# Carga de datos
ruta_datos = os.path.join("..", "data", "processed", "telecom_final.csv")

In [3]:
# Solo para comprobar que la ruta es correcta
try:
    df = pd.read_csv(ruta_datos)
    print(f"Datos cargados correctamente: {df.shape[0]} registros.")
    print("Columnas disponibles:", df.columns.tolist())
except FileNotFoundError:
    print("Error: No se encontró el archivo 'telecom_final.csv'. Verifica el paso anterior.")

Datos cargados correctamente: 7267 registros.
Columnas disponibles: ['ID_Cliente', 'Abandono', 'Genero', 'Jubilado', 'Pareja', 'Dependientes', 'Meses_Contrato', 'Servicio_Telefonico', 'Lineas_Multiples', 'Servicio_Internet', 'Seguridad_Online', 'Respaldo_Online', 'Proteccion_Dispositivo', 'Soporte_Tecnico', 'Streaming_TV', 'Streaming_Peliculas', 'Tipo_Contrato', 'Facturacion_Digital', 'Metodo_Pago', 'Cargo_Mensual', 'Cargo_Total', 'Gasto_Diario']


In [4]:
# Seleccionamos las variables de negocio (Tiempo y Dinero)
cols_numericas = ['Meses_Contrato', 'Cargo_Mensual', 'Cargo_Total', 'Gasto_Diario']

estadisticas = df[cols_numericas].describe().T

estadisticas['Rango'] = estadisticas['max'] - estadisticas['min']

display(estadisticas.round(2))

,count,mean,std,min,25%,50%,75%,max,Rango
Meses_Contrato,7267.0,32.35,24.57,0.00,9.00,29.00,55.00,72.00,72.00
Cargo_Mensual,7267.0,64.72,30.13,18.25,35.42,70.30,89.88,118.75,100.50
Cargo_Total,7267.0,2277.18,2268.65,0.00,396.20,1389.20,3778.52,8684.80,8684.80
Gasto_Diario,7267.0,2.16,1.00,0.61,1.18,2.34,3.00,3.96,3.35


In [5]:
# Análisis de frecuencia para variables categóricas
cols_categ = ['Genero', 'Pareja', 'Servicio_Telefonico', 'Tipo_Contrato', 'Metodo_Pago', 'Abandono']

display(df[cols_categ].astype(str).describe().T)

,count,unique,top,freq
Genero,7267,2,Masculino,3675
Pareja,7267,2,0,3749
Servicio_Telefonico,7267,2,1,6560
Tipo_Contrato,7267,3,Mensual,4005
Metodo_Pago,7267,4,Cheque electrónico,2445
Abandono,7267,3,0.0,5174


In [6]:
# Configuración de colores para las gráficas
COLORES = {
    'Principal': '#2E4053',  
    'Secundario': '#1ABC9C',  
    'Alerta': '#E74C3C',      
    'Neutro': '#95A5A6'       
}

In [7]:
# Histograma de Cargo Mensual
fig_hist = px.histogram(df, x='Cargo_Mensual', 
                        nbins=40, 
                        title='Distribución de Facturación Mensual',
                        marginal='box',
                        color_discrete_sequence=[COLORES['Principal']]) 

fig_hist.update_layout(
    xaxis_title="Cargo Mensual ($)",
    yaxis_title="Frecuencia (Clientes)",
    plot_bgcolor='rgba(0,0,0,0)', 
    bargap=0.1
)

fig_hist.show()

## Distribución de evasión


In [8]:
fig_pie = px.pie(df, names='Abandono', 
                 title='Proporción de Clientes: Retenidos vs Fuga',
                 color='Abandono',
                 color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']},
                 hole=0.5)

fig_pie.update_traces(textinfo='percent+label')
fig_pie.update_layout(
    annotations=[dict(text='Churn', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

fig_pie.show()

In [9]:
fig_bar_churn = px.histogram(df, x='Abandono', 
                             color='Abandono',
                             title='Cantidad de Clientes: Retenidos vs Fuga',
                             text_auto=True, 
                             color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']})

fig_bar_churn.update_layout(
    xaxis_title="Estado de Abandono (0=No, 1=Sí)",
    yaxis_title="Cantidad de Clientes",
    showlegend=False 
)

fig_bar_churn.show()

## Evasión por categorías

In [10]:
variables_analisis = ['Genero', 'Tipo_Contrato', 'Metodo_Pago', 'Servicio_Internet']

In [11]:
for col in variables_analisis:
    fig = px.histogram(df, x=col, 
                       color='Abandono', 
                       barmode='group', 
                       title=f'Evasión por {col}',
                       text_auto=True, 
                       color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']})
    
    fig.update_layout(
        xaxis_title=col,
        yaxis_title="Cantidad de Clientes",
        height=500,
        showlegend=True,
        bargap=0.15
    )
    
    fig.show()

## Conteo de evasión por variables numéricas

In [12]:
vars_numericas = ['Meses_Contrato', 'Cargo_Mensual', 'Cargo_Total']

#### Distribución numerica

In [13]:
for col in vars_numericas:
    fig = px.box(df, x='Abandono', y=col, 
                 color='Abandono',
                 title=f'Distribución de {col} según Estado del Cliente',
                 points="outliers", 
                 color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']})
    
    fig.update_layout(
        xaxis_title="¿Abandonó? (0=No, 1=Sí)",
        yaxis_title=col,
        height=500,
        showlegend=False 
    )
    
    fig.show()

#### Análisis de Fidelidad

In [14]:
fig_tenure = px.histogram(df, x="Meses_Contrato", color="Abandono",
                          title="¿Cuándo ocurren las fugas?",
                          barmode='overlay', 
                          nbins=72, 
                          opacity=0.7,
                          color_discrete_map={0: COLORES['Secundario'], 1: COLORES['Alerta']})

fig_tenure.update_layout(
    xaxis_title="Meses de Antigüedad",
    yaxis_title="Cantidad de Clientes",
    legend_title="Estado"
)

fig_tenure.show()

# **INFORME FINAL DEL PROYECTO** 

# Análisis de Evasión "Churn" - Telecom X

## 1. Introducción y Objetivos
**Telecom X** enfrenta un desafío crítico: una alta tasa de cancelación de clientes. 
El objetivo de este proyecto fue identificar los **factores determinantes** que impulsan a los usuarios a abandonar el servicio, utilizando datos históricos y herramientas de Data Science (Python, Pandas, Plotly).

## 2. Metodología: Limpieza y Tratamiento de Datos (ETL)
Para garantizar un análisis fiable, realizamos un riguroso proceso de ingeniería de datos:
1.  **Extracción:** Conexión a datos crudos en formato JSON anidado.
2.  **Transformación:**
    * Aplanado de estructuras complejas (`json_normalize`).
    * Corrección de tipos de datos (Conversión de `Total` a numérico).
    * **Traducción:** Renombramos todas las variables al español y estandarizamos valores (`Yes` -> `1`, `Male` -> `Masculino`).
    * **Ingeniería de Características:** Creamos la variable `Gasto_Diario` para medir el impacto cotidiano del precio.
3.  **Limpieza:** Imputación de valores nulos (asumiendo deuda 0 para clientes nuevos) y eliminación de duplicados.

## 3. Análisis Exploratorio de Datos (Hallazgos Clave)
A través de visualizaciones interactivas, detectamos los siguientes patrones:

* **Distribución de Evasión:** Existe un desbalance significativo. Un porcentaje considerable de clientes abandona, lo que valida la urgencia del problema.
* **Factor Económico:** Los clientes que se van (`Abandono=1`) tienen, en promedio, una **facturación mensual más alta** que los leales.
* **Factor Contractual:** La inmensa mayoría de las fugas provienen de contratos **Mensuales**. Los contratos a 1 o 2 años actúan como "escudos" efectivos contra el churn.
* **Antigüedad:** El "Valle de la Muerte" ocurre en los primeros meses. Si el cliente supera el primer año, su probabilidad de quedarse aumenta drásticamente.

## 4. Conclusiones e Insights
1.  **Sensibilidad al Precio:** Los usuarios de Telecom X son altamente sensibles al costo mensual. Métodos de pago como "Cheque Electrónico" (asociados a usuarios digitales/más volátiles) muestran mayor rotación.
2.  **El problema es el "Arranque":** La falta de compromiso a largo plazo (contratos mensuales) combinada con precios altos al inicio genera la tormenta perfecta para la cancelación temprana.
3.  **Tecnología:** Se detectó mayor rotación en usuarios de **Fibra Óptica** en comparación con DSL, sugiriendo posibles problemas de relación precio/calidad en ese servicio específico.

## 5. Recomendaciones Estratégicas
Basado en los datos, sugerimos al equipo de estrategia:
* **Plan de Retención "Onboarding":** Implementar descuentos agresivos o beneficios extra durante los primeros 6 meses para ayudar al cliente a cruzar el umbral de riesgo.
* **Migración de Contratos:** Incentivar a los usuarios "Mensuales" a pasar a contratos anuales mediante promociones (ej. "Paga 11 meses, recibe 12").
* **Auditoría de Fibra Óptica:** Revisar la competitividad de las tarifas y la estabilidad técnica del servicio de Fibra, ya que es el segmento con mayor fuga relativa.


